# Run a reconciliation

Click [HERE](magic-commands/Run a reconciliation.ipynb) for a notebook which is runnable in LUSID's JupyterHub using the 
`%%luminesce` magic command.
    

#### Step 1: Create transaction portfolio

```sql
-- ===============================================================
-- Description:
-- 1. In this query, we create two Transaction Portfolios in LUSID
-- ===============================================================

-- Defining scope and code variables
@@portfolioScope =

select 'LuminesceReconExample';

@@portfolioCode1 =

select 'UkEquityTracker';

@@portfolioCode2 =

select 'UkEquityActive';

@@base_currency =

select 'GBP';

@@created_date = select #2000-01-01#;

-- Define the portfolio data
@create_portfolio =

select 'Transaction' as PortfolioType, @@portfolioScope as PortfolioScope, @@portfolioCode1 as
   PortfolioCode, @@portfolioCode1 as DisplayName, '' as Description, @@created_date as Created,
   '' as SubHoldingKeys, @@base_currency as BaseCurrency

union all

values (
   'Transaction', @@portfolioScope, @@portfolioCode2, @@portfolioCode2, '', @@created_date, '',
   @@base_currency
   );

-- Upload the portfolio into LUSID
@response_create_portfolio =

select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

select *
from @response_create_portfolio;

```

#### Step 2: Create instruments

```sql
-- =====================================================
-- Description:
-- 1. In this query, we  load instruments from an Excel
-- file into LUSID
-- =====================================================

@@file_date =

select strftime('20221101');

@instruments_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=instruments
--addFileName
enduse;

-- Run instruments transformation
@instruments_for_upload =

select Ticker, Name as DisplayName, ISIN as Isin, 'EQ' || Isin as ClientInternal, SEDOL as Sedol,
   Currency as DomCcy, 'Equities' as AssetClass, 'Equities' as SimpleInstrumentType
from @instruments_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @instruments_for_upload;

```

#### Step 3: Create holdings

```sql
-- =====================================================
-- Description:
-- 1. In this query, we load holdings into LUSID for the
-- two portfolios.
-- =====================================================

@@file_date =

select strftime('20221101');

@@portfolioScope =

select 'LuminesceReconExample';

@holdings_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=lusid_holdings
--addFileName
enduse;

-- Run holdings transformation
@holdings_for_upload =

select portfolio as [PortfolioCode], @@portfolioScope as [PortfolioScope], holding_date as
   [EffectiveAt], 'EQ' || InstrumentId as [ClientInternal], units as [Units], Currency as
   CostCurrency
from @holdings_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Portfolio.Holding.Writer
where ToWrite = @holdings_for_upload;

```

#### Step 4: Create reconciliation view

```sql
-- ============================================================
-- Description:
-- 1. In this query, we create a view to run the reconciliation
-- ============================================================


@recon_view =

use Sys.Admin.SetupView
--provider=Test.Example.HoldingsRecon

--parameters
file_name,Text,equity_holdings_20221101.xlsx,true
portfolio,Text,UkEquityTracker,true
scope,Text,LuminesceReconExample,true
recon_date,Date,2022-03-01,true
----

@@file_name = select #PARAMETERVALUE(file_name);
@@portfolio = select #PARAMETERVALUE(portfolio);
@@scope = select #PARAMETERVALUE(scope);
@@recon_date = select #PARAMETERVALUE(recon_date);


@ext_holdings_from_excel =

use Drive.Excel with @@file_name
--file=/luminesce-examples/{@@file_name}
--worksheet=ext_holdings
--addFileName
enduse;

-- Get LUSID holdings

@holdings_from_lusid = select
h.PortfolioCode,
i.DisplayName as [InstrumentName],
i.LusidInstrumentId,
i.ClientInternal,
i.Isin,
h.HoldingType,
h.Units
from Lusid.Portfolio.Holding h
join Lusid.Instrument i on (i.LusidInstrumentId = h.LusidInstrumentId)
where h.PortfolioScope = @@scope
and h.PortfolioCode = @@portfolio
and h.EffectiveAt = @@recon_date
order by h.HoldingType desc;

-- Run a reconciliation

select
l.PortfolioCode as [Left_PortfolioCode],
l.InstrumentName as [Left_InstrumentName],
l.Isin as [Left_Isin],
l.HoldingType as [Left_HoldingType],
l.Units as [Left_Units],
r.Units as [Right_Units],
(r.Units - l.Units) as [Units_Diff],
round(((r.Units *1.0) / l.Units) -1 ,5) as [Units_Diff_Pct]
from @holdings_from_lusid l
join @ext_holdings_from_excel r on (l.ClientInternal = 'EQ' || r.InstrumentId
and r.portfolio = @@portfolio);

enduse;


-- wait for view/provider to be created on the grid

select * from @recon_view wait 20;

```

#### Step 5: Run recon with notifications

```sql
-- =====================================================
-- Description:
-- 1. In this query, we create event notifications from
-- the reconciliation results
-- =====================================================

-- define strings for a pass and fail reconciliation

@@failed_event =

select 'HoldingReconFailed';

@@passed_event =

select 'HoldingReconPassed';

-- Count the number of breaks from thr reconciliation
-- We use an absolute count to assess if there are
-- any breaks or not

@recon_result =

select sum(abs(Units_Diff)) as [Breaks]
from Test.Example.HoldingsRecon
where portfolio = 'UkEquityActive';

-- If there are breaks, we generate a failed event
-- If there are no breaks, we generate a passed event

@event =

select iif(Breaks = 0, @@passed_event, @@failed_event) as [EventToGenerate]
from @recon_result;

-- Generate the event in LUSID

@@event_to_generate =

select EventToGenerate
from @event;

@event_details =

select 'Manual' as [EventType], @@event_to_generate as [Message], ('Holdings recon: ' || @@event_to_generate
      ) as [Subject], date () as [EventTime];

select *
from Notification.Event.Writer
where ToWrite = @event_details;
```